## Task 2: Words as Features

In [6]:
import gzip
import json
from sklearn.feature_extraction.text import CountVectorizer

with gzip.open('dataset/goemotions.json.gz', 'r') as file:
    fileData = file.read()
    dataset = json.loads(fileData.decode('utf-8'))

posts = []

# Extracting posts into one list
for i in range(len(dataset)):
    posts.append(dataset[i][0])
    
vectorizer = CountVectorizer()
vectorizer.fit(posts)
 
# Printing the number of tokens (size of the vocabulary)
print("Number of tokens (size of vocabulary) in the dataset:", len(vectorizer.vocabulary_))

Number of tokens (size of vocabulary) in the dataset:  30449


### 2.1 Extracting tokens/words and their frequencies